# Yapay Sinir Ağları (Artificial Neural Networks)

In [1]:
import torch # PyTorch kütüphanesi , tensor işlemleri
import torch.nn as nn # Yapay sinir ağı katmanlarını tanımlamak için kullanılır
import torch.optim as optim # Optimizasyon algoritmalarını içeren modül
import torchvision # Görüntü işleme ve pre-defined modelleri içerir
import torchvision.transforms as transforms # Görüntü dönüşümlerini yapmak
import matplotlib.pyplot as plt # Görselleştirme

Optional : Cihaz belirleme 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

**Problem Tanımı:** MNIST veri seti ile rakam sınıflandırma projesi

### 1)Veri Setinin Yüklenmesi

In [3]:
# batch_size : Her iterasyonda işlenecek veri miktarı .
def get_data_loaders(batch_size = 64):
    
    transform = transforms.Compose([
        transforms.ToTensor() , # Görüntüyü tensore çevirir ve 0-255 -> 0-1 olçeklenir
        transforms.Normalize((0.5,),(0.5,)) # Piksel değerlerini -1 ile +1 arasinda olacak şekilde ölçekler 
    ])

    train_set = torchvision.datasets.MNIST(root = "./mnist_data" , train = True , download = True , transform = transform )
    test_set = torchvision.datasets.MNIST(root = "./mnist_data" , train = False , download = True , transform = transform )

    # PyTorch veri yükleyicisi oluşturma 
    train_loader = torch.utils.data.DataLoader(train_set , batch_size = batch_size , shuffle = True )
    test_loader = torch.utils.data.DataLoader(test_set , batch_size = batch_size , shuffle = False )
    
    return train_loader , test_loader

In [4]:
train_loader , test_loader = get_data_loaders()

### 2)Veri Görselleştirme

In [5]:
def visualize_samples(loader , n):
    images , labels = next(iter(loader)) # İlk batch den görüntü ve etiketleri alalım
    fig , axes = plt.subplots(1 , n , figsize = (10,5)) # n farklı görüntü için görselleştirme alanı
    for i in range(n):
        axes[i].imshow(images[i].squeeze() , cmap = "gray") # Görseli gri tonlarda göster
        axes[i].set_title(f"Label : {labels[i].item()}") # Görüntüye ait etiketi başlık olarak yaz
        axes[i].axis("off") # Eksenleri gizle
    plt.show()

In [6]:
# visualize_samples(train_loader , 4)

### 3)Modelin Oluşturulması

In [7]:
# Yapay sinir ağı Sınıfı
class NeuralNetwork(nn.Module): # PyTorch'un nn.Module sınıfından miras alıyor
    
    # NN inşaa etmek için gerekli olan bileşenleri tanımla  
    def __init__(self):
        super(NeuralNetwork , self).__init__()
        # Elimizde bulunan görüntüleri vektör haline çevirelim.(2D => 1D)
        self.flatten = nn.Flatten()
        
        # İlk tam bağlı katmanı oluşturma
        self.fc1 = nn.Linear(28*28 , 128)
        
        # Aktivasyon fonk. oluştur 
        self.relu = nn.ReLU() 
        
        # İkinci tam bağlı katmanı oluştur 
        self.fc2 = nn.Linear(128 , 64)
        
        # Çıktı katmanını oluştur : 64 => input size , 10 => output size (0-9 etiketleri)
        self.fc3 = nn.Linear(64 , 10)

    # Forward Propagation : İleri yayılım , giriş olarak x = görüntü alsın
    def forward(self , x):
        x = self.flatten(x) # initial x = 28*28 lik bir görüntü -> düzleştir 784 vektör haline getir
        x = self.fc1(x) # Birinci bağlı katman
        x = self.relu(x) # Aktivasyon fonk.
        x = self.fc2(x) # İkinci bağlı katman
        x = self.relu(x) # Aktivasyon fonk.
        x = self.fc3(x) # Output katmanı

        # Modelin çıktısı
        return x

**Create and Compile**

In [8]:
model = NeuralNetwork().to(device)

# Kayıp fonk. ve optimizasyon algoritması belirleme
define_loss_and_optimizer = lambda mode : (
    nn.CrossEntropyLoss() , # Multi Class Classification problems loss func.
    optim.Adam(model.parameters() , lr = 0.001) # Update weights with adam
)
criterion , optimizer = define_loss_and_optimizer(model)

**Modelin Eğitimi**

In [9]:
# criterion : Loss fonk

def train_model(model , train_loader , criterion , optimizer , epochs = 10 ):
    
    # Modeli eğitim moduna alalım
    model.train()
    
    # Her bir epoch sonucunda elde edilen loss değerlerini saklamak için bir liste
    train_losses = []

    # Belirtilen epochs sayısı kadar eğitim yapalım
    for epoch in range(epochs):
        total_loss = 0
        
        # Tüm eğitim verileri üzerinde iterasyon gerçekleştir
        for images , labels in train_loader:
            images , labels = images.to(device) , labels.to(device) # Verileri cihaza taşı
            
            # Gradyanları sıfırla
            optimizer.zero_grad()
            
            # Modeli Uygula , forward pro. 
            predictions = model(images)
            
            # Loss hesaplama -> y_pred , y_real
            loss = criterion(predictions,labels)
            
            # Geri yayılım yani gradyan hesaplama
            loss.backward()
            
            # Update weight (ağırlık güncelle)
            optimizer.step()

            total_loss = total_loss + loss.item()

        avg_loss = total_loss / len(train_loader) # Ortalama kayıp hesaplama
        train_losses.append(avg_loss)
        print(f"Epochs : {epoch+1} , Loss : {avg_loss:.3f}")
    # Loss Graph
    # plt.figure()
    # plt.plot(range( 1 , epochs + 1) ,train_losses , marker = "o" , linestyle = "-" , label = "Train Loss")
    # plt.xlabel("Epochs")
    # plt.ylabel("Loss")
    # plt.title("Training Loss")
    # plt.legend()
    # plt.show()

In [10]:
train_model(model , train_loader , criterion , optimizer , epochs = 10)

Epochs : 1 , Loss : 0.406
Epochs : 2 , Loss : 0.191
Epochs : 3 , Loss : 0.140
Epochs : 4 , Loss : 0.114
Epochs : 5 , Loss : 0.097
Epochs : 6 , Loss : 0.085
Epochs : 7 , Loss : 0.075
Epochs : 8 , Loss : 0.070
Epochs : 9 , Loss : 0.060
Epochs : 10 , Loss : 0.055


### 4)Modelin Test Edilmesi

In [13]:
def test_model(model , test_loader):
    model.eval() # Modeli değerlendirme moduna al  
    correct = 0 # Doğru tahmin sayacı
    total = 0 # Toplam veri sayacı

    with torch.no_grad(): # Gradyan hesaplama gereksiz olduğundan kapattık
        for images , labels in test_loader: # Test veri kümesini dögüye al
            images , labels = images.to(device) , labels.to(device) # Verileri cihaza taşı
            predictions = model(images) 
            _ , predicted = torch.max(predictions , 1) # En yüksel olasılıklı sınıfın etiketini bul
            total += labels.size(0) # Toplam veri sayısını güncelle
            correct += (predicted == labels).sum().item() # Doğru tahmin say
    print(f"Test Accuracy : {100 * correct / total:.3f}%")

In [14]:
test_model(model , test_loader)

Test Accuracy : 97.270%
